# Generate the systems for multistates

In [1]:
#IMPORT
import os, sys, glob
import time

from collections import defaultdict
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX
from pygromos.files.simulation_parameters import imd
from pygromos.data import solvent_coordinates as solvents

from pygromos.files.coord import cnf
from pygromos.files.blocks.coord_blocks import GENBOX, Pbc
from pygromos.files.blocks import imd_blocks

import restraintmaker

#CHANGE HERE
gromosPP_bin_path = "/home/bschroed/Documents/code/gromosPP/installed/bin"
gromosXX_bin_path = "/home/bschroed/Documents/code/gromosXX/installed/bin"
restraintmaker_path = os.path.abspath(os.path.dirname(restraintmaker.__file__)+"/..")

## Path definitions
generate the subfolders for the systems - no changes required here.

In [2]:
sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
pairwise_dir = sets_dir+"/pairwise"

resn_lib_path = sets_dir+"/resn_lib.lib"


gromPP = GromosPP(gromosPP_bin_path)
gromXX = GromosXX(gromosXX_bin_path)

atb_dirs = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/ATB_molecules"
sys.path.append(atb_dirs+"/..")

##get all_single file_tops:
all_tops = glob.glob(atb_dirs+"/*/*top")
state_all_tops={os.path.basename(value).split(".")[0]: value for value in all_tops}



## Build gromos files

In [3]:
import time
from pygromos.utils import bash
import utils_test_set_ATB as utils
all_states = utils.multistate_ligand_sets["all"]
all_combos = [x for x in os.listdir(pairwise_dir) if(os.path.isdir(pairwise_dir+"/"+x))]

all_combos = [x if(len(x)==2) else (x[0], "_"+x[2]) for x in list(map(lambda x: x.split("_"), all_combos))]
print(all_combos)
print()
all_ligs = ["_O6T", "G277", "M097", "6KET", "F313"]
from itertools import combinations
all_combos = list(combinations(all_ligs,2))

all_combos = [["M097", "G277"], ["G277", "M097"], ["G277", "6KET"], ["F313", "_O6T"], ["_O6T", "F313"], ["_O6T", "6KET"], ["6KET", "M097"], ["6KET", "F313"], ["6KET", "_O6T"],["6KET", "G277"]]
all_combos = [("M097", "G277"), ("G277", "6KET"), ("_O6T", "F313"), ("_O6T", "6KET"), ("6KET", "F313")]

for state_a, state_b in all_combos:
    system_tops = [state_all_tops[x] for x in [state_a, state_b]]

    name = state_a+"_"+state_b
    out_dir = pairwise_dir+"/"+name
    out_prefix_path = out_dir+"/"+name
    print(name, [state_a, state_b])

    if(not os.path.isdir(out_dir)):
        bash.make_folder(out_dir)
    #build dualTop system
    out_top_path, out_ptp_path = gromPP.prep_eds(in_top_paths=system_tops, number_of_eds_states=len(system_tops), out_file_path=out_prefix_path)

    #generate cnf
    print(glob.glob(out_dir+"/*.pdb"))
    in_pdb = glob.glob(out_dir+"/*.pdb")[0]
    out_cnf_path = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=out_top_path, in_lib_path=resn_lib_path,
                                     out_cnf_path=out_prefix_path+".cnf")

    ##build box
    box_cnf = cnf.Cnf(out_cnf_path)
    box_cnf.supress_atomPosition_singulrarities()
    box_cnf.add_block(block=GENBOX(pbc=Pbc.rectangular,
                            length=[3.610220118 for x in range(3)],
                            angles=[90 for x in range(3)],
                            euler=[0 for x in range(3)],
                            origin=[0 for x in range(3)]))
    box_cnf_path = box_cnf.write(out_cnf_path.replace(".cnf", "_box.cnf"))

    #build posres/refpos
    refpos_path = box_cnf.write_refpos(out_prefix_path+".rfp")
    posresspec_path = box_cnf.write_possrespec(out_prefix_path+".por", residues=list(box_cnf.residues.keys()))

    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_cnf_path, periodic_boundary_condition="v",
                out_file_path=out_prefix_path+"_box.pdb", out_file_format="pdb", time=0, verbose=True)

    #solvate
    out_solv_cnf_path=out_prefix_path+"_spc.cnf"
    gromPP.sim_box(in_top_path=out_top_path, minwall=None, in_cnf_path=box_cnf_path, in_solvent_cnf_file_path=solvents.spc,
                   out_cnf_path=out_solv_cnf_path, periodic_boundary_condition="r cog", gathering_method=None,  boxsize=True, verbose=True)
    time.sleep(2)

    gromPP.frameout(in_top_path=out_top_path, in_coord_path=out_solv_cnf_path, periodic_boundary_condition="r cog",
                    out_file_path=out_prefix_path+"_solv.pdb", include="all", out_file_format="pdb", time=0)

[('M030', '_O71'), ['M030', 'F313'], ['M030', 'G277'], ('', '_F313'), ('', '_M097'), ['M030', 'E1VB'], ['M097', '6KET'], ['M097', 'F313'], ['6KET', 'F313'], ['G277', 'M097'], ('', '_6KET'), ('', '_G277'), ['6J29', 'G078'], ['M030', 'M097'], ['M030', 'G078'], ['G277', 'F313'], ['M030', 'S002'], ('M030', '_O70'), ('M030', '_O6T'), ['M097', 'G277'], ['M030', '6J29'], ['M030', '8018'], ['G277', '6KET'], ['M030', 'TVVS'], ['M030', '6KET'], ['M030', 'G209'], ['M030', 'M218'], ['8018', 'G078'], ['8018', '6J29'], ('M030', '_P8I')]

M097_G277 ['M097', 'G277']
['/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/pairwise/M097_G277/M097_G277.pdb', '/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/pairwise/M097_G277/M097_G277_box.pdb']
Cnf
gromosPP.frameout: command:
/home/bschroed/Documents/code/gromosPP/installed/bin/frameout @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts